# AI기법과 활용 - Week 08
TF-IDF와 Random Forest를 활용한 감성분석
____

## 1. 데이터 다운로드

In [ ]:
import sys
!{sys.executable} -m pip install konlpy
!pip install konlpy

In [ ]:
!wget -O ratings_train.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -O ratings_test.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

## 2. 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.read_csv("ratings_train.txt", sep="\t")
df_test = pd.read_csv("ratings_test.txt", sep="\t")
df_train.head(n=10)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [ ]:
# 일단 너무 오래걸려서 10000개만 갖고 훈련을 진행하겠습니다.
df_train = df_train.sample(10000)

## 3. Tokenizer 정의

In [ ]:

pd.options.mode.chained_assignment = None
np.random.seed(0)

from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
stop_words = []
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun", "Verb"], stopword=stop_words+[]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 테스트 문장
rawdata = df_train['document'].tolist()

## 4. TF-IDF 계산 

In [ ]:

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1, #TODO : 특정 단어가 최소 등장해야하는 문서의 수 = 이 이하 등장 하는 단어는 무시
    max_df=1.0, #TODO : 특정 단어가 최대 등장해야하는 문서의 수 = 이 이상 등장 하는 단어는 무시
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

print(
    'fit_transform, (review {}, feature {})'.format(X.shape[0], X.shape[1])
)




In [ ]:

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=0.001, #TODO : 특정 단어가 최소 등장해야하는 문서의 수 = 이 이하 등장 하는 단어는 무시
    max_df=0.999, #TODO : 특정 단어가 최대 등장해야하는 문서의 수 = 이 이상 등장 하는 단어는 무시
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

print(
    'fit_transform, (review {}, feature {})'.format(X.shape[0], X.shape[1])
)




In [ ]:

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=0.01, #TODO : 특정 단어가 최소 등장해야하는 문서의 수 = 이 이하 등장 하는 단어는 무시
    max_df=0.99, #TODO : 특정 단어가 최대 등장해야하는 문서의 수 = 이 이상 등장 하는 단어는 무시
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

print(
    'fit_transform, (review {}, feature {})'.format(X.shape[0], X.shape[1])
)




In [ ]:

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=5, #TODO : 특정 단어가 최소 등장해야하는 문서의 수 = 이 이하 등장 하는 단어는 무시
    max_df=0.9, #TODO : 특정 단어가 최대 등장해야하는 문서의 수 = 이 이상 등장 하는 단어는 무시
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

print(
    'fit_transform, (movie {}, feature {})'.format(X.shape[0], X.shape[1])
)




In [ ]:
print(X.toarray())



# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()


In [ ]:
y = df_train['label'].to_numpy()

## 5. Random Forest로 훈련

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X, y)



In [ ]:
## 테스트 데이터셋으로 평가

In [ ]:
rawdata = df_test['document'].tolist()
print("Preprocessing...")
test_X = vectorize.transform(rawdata)
test_y = df_test['label'].to_numpy()
print("Scoring...")
clf.score(test_X, test_y)

## 6.직접 테스트

In [ ]:
text = "최악의 연기"
x = vectorize.transform([text])
print(clf.predict(x))
print(clf.predict_proba(x))

In [ ]:
text = "최고의 연기"
x = vectorize.transform([text])
print(clf.predict(x))
print(clf.predict_proba(x))

In [ ]:
text = "진짜 재미없는 영화"
x = vectorize.transform([text])
print(clf.predict(x))
print(clf.predict_proba(x))

In [ ]:
text = "진짜 재미있는 영화"
x = vectorize.transform([text])
print(clf.predict(x))
print(clf.predict_proba(x))